In [ ]:
import numpy as np
import torch

import neural_network
from neural_network import sigmoid, sigmoid_der, ReLU, ReLU_der, LeakyReLU, LeakyReLU_der, identity, identity_der, mse, mse_der
import data_sets
import optimizers

In [2]:
# Adam, Runge

np.random.seed(0)

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_runge_data(100)

    custom_net = neural_network.NeuralNetwork(1, [50, 50, 1], [LeakyReLU, sigmoid, identity], [LeakyReLU_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.Adam(0.01), 1_000)
    print(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(torch_net.parameters(), lr=0.01)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")


0.012716826068278547
0.014786051586270332
-----
0.008892519406885372
0.008875338360667229
-----
0.009796070760283701
0.011496654711663723
-----
0.011778309761858629
0.011730866506695747
-----
0.007380880139930169
0.007425447925925255
-----


In [3]:
# RMSprop, Runge

np.random.seed(1)

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_runge_data(100)

    custom_net = neural_network.NeuralNetwork(1, [50, 50, 1], [ReLU, sigmoid, identity], [ReLU_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.RMS_prop(0.0005), 1_000)
    print(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(1, 50),
        torch.nn.ReLU(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.RMSprop(torch_net.parameters(), lr=0.0005, alpha=0.9)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")


0.010802065486567263
0.010071261785924435
-----
0.007000551151695799
0.01337449811398983
-----
0.010364383193260867
0.010889648459851742
-----
0.009595088514789149
0.00858983863145113
-----
0.009284913047182767
0.01496945507824421
-----


In [ ]:
# Constant, Gaussian 2D

np.random.seed(0)

for _ in range(5):
    x_train, x_test, y_train, y_test = data_sets.load_gaussian2D_data(100)

    custom_net = neural_network.NeuralNetwork(2, [50, 50, 1], [sigmoid, sigmoid, identity], [sigmoid_der, sigmoid_der, identity_der], mse, mse_der)
    custom_net.train_network(x_train, y_train, 1, optimizers.Constant(0.005), 1_000)
    print(custom_net.cost(x_test, y_test))

    torch_net = torch.nn.Sequential(
        torch.nn.Linear(2, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 50),
        torch.nn.Sigmoid(),
        torch.nn.Linear(50, 1)
    )
    criterion = torch.nn.MSELoss()
    # torch.optim has no 'Constant' optimizer. Use SGD with a fixed learning rate instead.
    optimizer = torch.optim.SGD(torch_net.parameters(), lr=0.005)
    for _ in range(1_000):
        inputs = torch.from_numpy(x_train.astype(np.float32))
        labels = torch.from_numpy(y_train.astype(np.float32))
        optimizer.zero_grad()
        outputs = torch_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    torch_net.eval()
    with torch.no_grad():
        print(criterion(torch_net(torch.from_numpy(x_test.astype(np.float32))), torch.from_numpy(y_test.astype(np.float32))).item())
    print("-----")


0.05681639303350236
0.08369176089763641
-----
0.030004641960032986
0.0607672818005085
-----
0.024899037347751705
0.028699608519673347
-----
0.06578194880644363
0.1222381517291069
-----
0.04480216509681431
0.05150032043457031
-----
